# **Redes Hopfield**

In [5]:
import polars as pl
import numpy as np
from scipy.optimize import linear_sum_assignment

# Configuración inicial
cities = ['A', 'B', 'C', 'D', 'E']
n = len(cities)
city_index = {c: i for i, c in enumerate(cities)}

# Matriz de distancias en Polars
dist_data = {
    'A': [0, 5, 5, 6, 4],
    'B': [5, 0, 3, 7, 6],
    'C': [5, 3, 0, 4, 8],
    'D': [6, 7, 4, 0, 5],
    'E': [4, 6, 8, 5, 0]
}

dist_df = pl.DataFrame(dist_data).select(
    pl.all().cast(pl.Float64)
)

# Parámetros del modelo
PRF = 2000
PRC = 2000
PD = 50
dt = 0.01
max_iter = 10000

# Inicialización de la matriz V en Polars
np.random.seed(42)
V_np = np.random.rand(n, n) * 0.1 + 0.1
V = pl.DataFrame(V_np, schema=[f"pos_{i}" for i in range(n)])

print("Iniciando simulación...")

# Simulación principal
for iteration in range(max_iter):
    # Cálculo de restricciones con Polars
    # Suma por filas (cada neurona debe tener exactamente una activación)
    V_np_current = V.to_numpy()
    sum_rows_np = np.sum(V_np_current, axis=1, keepdims=True) - V_np_current
    sum_rows = pl.DataFrame(sum_rows_np, schema=V.columns)

    # Suma por columnas (cada ciudad debe ser visitada exactamente una vez)
    sum_cols_np = np.sum(V_np_current, axis=0, keepdims=True) - V_np_current
    sum_cols = pl.DataFrame(sum_cols_np, schema=V.columns)

    # Desplazamientos cíclicos usando NumPy
    V_prev_np = np.roll(V_np_current, shift=1, axis=1)
    V_next_np = np.roll(V_np_current, shift=-1, axis=1)

    # Cálculo de distancias con NumPy
    sum_dist_np = dist_df.to_numpy() @ (V_prev_np + V_next_np)
    sum_dist = pl.DataFrame(sum_dist_np, schema=V.columns)

    # Actualización de la matriz V
    U_np = sum_rows.to_numpy() * (-PRF) + sum_cols.to_numpy() * (-PRC) + sum_dist.to_numpy() * (-PD)
    V_np_new = V_np_current + U_np * dt

    # Función de activación sigmoide
    V_np_sigmoid = 1 / (1 + np.exp(-V_np_new))
    V = pl.DataFrame(V_np_sigmoid, schema=V.columns)

    # Mostrar progreso cada 1000 iteraciones
    if iteration % 1000 == 0:
        print(f"Iteración {iteration}/{max_iter}")

print("Simulación completada. Procesando resultados...")

# Binarización y resultados finales
V_np_final = V.to_numpy()
print("Matriz final V:")
print(V_np_final)

# Usar linear_sum_assignment para encontrar la mejor asignación
row_ind, col_ind = linear_sum_assignment(-V_np_final)

# Crear el tour basado en la asignación
tour_indices = col_ind[np.argsort(row_ind)]
tour = [cities[i] for i in tour_indices]

print(f"\nAsignación encontrada:")
for i, j in zip(row_ind, col_ind):
    print(f"Ciudad {cities[i]} -> Posición {j}")

print(f"\nRecorrido encontrado: {tour}")

# Verificación de validez
if len(set(tour)) == n:
    print("✓ Recorrido válido: todas las ciudades visitadas una vez")

    # Calcular distancia total del tour
    total_distance = 0
    for i in range(n):
        current_city = city_index[tour[i]]
        next_city = city_index[tour[(i + 1) % n]]
        distance = dist_data[cities[current_city]][next_city]
        total_distance += distance
        print(f"{tour[i]} -> {tour[(i + 1) % n]}: distancia = {distance}")

    print(f"\nDistancia total del recorrido: {total_distance}")
else:
    print("✗ Solución no válida, ajustar parámetros")
    print(f"Ciudades únicas en el tour: {len(set(tour))}/{n}")

# Mostrar matriz de valores finales más legible
print(f"\nMatriz V final (redondeada a 3 decimales):")
V_rounded = np.round(V_np_final, 3)
for i, city in enumerate(cities):
    print(f"{city}: {V_rounded[i]}")

Iniciando simulación...
Iteración 0/10000
Iteración 1000/10000
Iteración 2000/10000
Iteración 3000/10000
Iteración 4000/10000
Iteración 5000/10000
Iteración 6000/10000
Iteración 7000/10000
Iteración 8000/10000
Iteración 9000/10000
Simulación completada. Procesando resultados...
Matriz final V:
[[0.5 0.5 0.5 0.5 0.5]
 [0.5 0.5 0.5 0.5 0.5]
 [0.5 0.5 0.5 0.5 0.5]
 [0.5 0.5 0.5 0.5 0.5]
 [0.5 0.5 0.5 0.5 0.5]]

Asignación encontrada:
Ciudad A -> Posición 0
Ciudad B -> Posición 1
Ciudad C -> Posición 2
Ciudad D -> Posición 3
Ciudad E -> Posición 4

Recorrido encontrado: ['A', 'B', 'C', 'D', 'E']
✓ Recorrido válido: todas las ciudades visitadas una vez
A -> B: distancia = 5
B -> C: distancia = 3
C -> D: distancia = 4
D -> E: distancia = 5
E -> A: distancia = 4

Distancia total del recorrido: 21

Matriz V final (redondeada a 3 decimales):
A: [0.5 0.5 0.5 0.5 0.5]
B: [0.5 0.5 0.5 0.5 0.5]
C: [0.5 0.5 0.5 0.5 0.5]
D: [0.5 0.5 0.5 0.5 0.5]
E: [0.5 0.5 0.5 0.5 0.5]
